In [10]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import time

### Puzzle Functions: `available_actions` and `do_action`

1. **`available_actions(state)`**:
   - Determines the valid moves in the puzzle given the current `state`.
   - Identifies the position of the empty tile (value `0`) and returns a list of possible `Action` objects that move the empty tile up, down, left, or right, respecting the puzzle boundaries.

2. **`do_action(state, action)`**:
   - Executes a given `Action` on the puzzle `state`.
   - Swaps the empty tile's position (`action.pos1`) with the target tile's position (`action.pos2`), returning the updated state.


In [3]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

In [4]:
def print_solution_steps(start_state, solution_path):
    """Prints each state of the grid starting from the initial state, following the moves in the solution."""
    current_state = start_state.copy()
    print("Initial state:")
    print(current_state)
    print("\n---")

    for step, move in enumerate(solution_path, 1):
        # Perform the move
        current_state = do_action(current_state, move)

        # Print the updated state
        print(f"Step {step}: Move the tile from {move.pos2} to {move.pos1}")
        print(current_state)
        print("\n---")


### Randomizing the Puzzle State

In this section, we define the puzzle dimension and create a named tuple `action` to represent moves. We then generate a random initial state for the puzzle by performing a series of random moves. The goal state is also defined for comparison during the solving process.

- `PUZZLE_DIM = 3`: Defines the dimension of the puzzle (3x3).
- `action`: A named tuple to represent the positions involved in a move.
- `RANDOMIZE_STEPS = 1_000`: Number of random moves to generate the initial state.
- `state`: The initial state of the puzzle after randomization.
- `goal`: The goal state of the puzzle.

The randomization process involves performing a series of random valid moves on the initial ordered state.


In [6]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])
# Generare uno stato casuale
RANDOMIZE_STEPS = 1_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

goal = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))


Randomizing:   0%|          | 0/1000 [00:00<?, ?it/s]

### Best First Search

### `NPuzzleSolver`: A Solver for the N-Puzzle Problem

This class implements a solver for the N-Puzzle problem, leveraging **Greedy Best-First Search** and a heuristic based on Manhattan distance.

#### **Attributes**
- `start_state`: The initial state of the puzzle.
- `goal_state`: The target state to achieve.
- `n`: The dimension of the puzzle (e.g., 3 for a 3x3 puzzle).

#### **Methods**
1. **`is_goal(state)`**:
   - Checks if the current `state` matches the `goal_state`.

2. **`heuristic(state)`**:
   - Computes the **Manhattan distance**, summing the distances of each tile from its target position.
   - Ignores the empty tile (value `0`).

3. **`greedy_search()`**:
   - Implements **Greedy Best-First Search**:
     - Expands states based on the heuristic value (Manhattan distance).
     - Keeps track of explored states to avoid revisiting them.
   - Returns:
     - `solution_path`: A sequence of actions leading to the solution.
     - `cost`: The total number of states expanded.

4. **Execution Flow**:
   - Start with the initial puzzle state and iterate until:
     - The goal is reached (prints the solution path, quality, cost, and efficiency).
     - No solution is found (prints a failure message).


In [11]:

class NPuzzleSolver:
    def __init__(self, start_state, goal_state):
        self.start_state = start_state
        self.goal_state = goal_state
        self.n = PUZZLE_DIM
    
    def is_goal(self, state):
        return np.array_equal(state, self.goal_state)
    
    def heuristic(self, state):
        """Calculate Manhattan distance."""
        distance = 0
        for i in range(self.n):
            for j in range(self.n):
                value = state[i, j]
                if value != 0:  # Skip the empty tile
                    goal_x, goal_y = divmod(value - 1, self.n)
                    distance += abs(i - goal_x) + abs(j - goal_y)
        return distance

    def greedy_search(self):
        """Greedy Best-First Search."""
        open_list = [(self.start_state, self.heuristic(self.start_state), [])]  # (state, heuristic, path)
        closed_set = set()
        expanded_states = 0  # Counter for the number of states evaluated
        
        while open_list:
            # Sort the open list by heuristic value (ascending)
            open_list.sort(key=lambda x: x[1])
            
            # Get the state with the smallest heuristic
            current_state, _, path = open_list.pop(0)
            
            # Increment the counter
            expanded_states += 1
            
            # Check if we've reached the goal
            if self.is_goal(current_state):
                print("Goal reached!")
                return path, expanded_states  # Return the sequence of moves and cost
            
            # Add current state to the closed set
            closed_set.add(tuple(map(tuple, current_state)))  # Convert array to tuple
            
            # Expand neighbors
            for move in available_actions(current_state):
                neighbor = do_action(current_state, move)
                neighbor_tuple = tuple(map(tuple, neighbor))
                if neighbor_tuple not in closed_set:
                    open_list.append((neighbor, self.heuristic(neighbor), path + [move]))
        
        print("No solution found.")
        return None, expanded_states

start_time = time.time()
solver = NPuzzleSolver(state, goal)

print("Initial State:")
print(state)

solution_path, cost = solver.greedy_search()
end_time = time.time()
print(f"Execution time: {end_time - start_time:.4f} seconds")

if solution_path:
    print("Steps to solve the puzzle:")
    print_solution_steps(state, solution_path)

    # Quality and Cost
    quality = len(solution_path)
    print(f"\nQuality (number of actions in the solution): {quality}")
    print(f"Cost (total number of actions evaluated): {cost}")
    print(f"Efficiency (Quality vs Cost): {quality / cost:.4f}")

else:
    print("No solution found.")



Initial State:
[[2 5 6]
 [8 0 7]
 [4 1 3]]
Goal reached!
Execution time: 0.0151 seconds
Steps to solve the puzzle:
Initial state:
[[2 5 6]
 [8 0 7]
 [4 1 3]]

---
Step 1: Move the tile from (1, 0) to (1, 1)
[[2 5 6]
 [0 8 7]
 [4 1 3]]

---
Step 2: Move the tile from (2, 0) to (1, 0)
[[2 5 6]
 [4 8 7]
 [0 1 3]]

---
Step 3: Move the tile from (2, 1) to (2, 0)
[[2 5 6]
 [4 8 7]
 [1 0 3]]

---
Step 4: Move the tile from (1, 1) to (2, 1)
[[2 5 6]
 [4 0 7]
 [1 8 3]]

---
Step 5: Move the tile from (1, 2) to (1, 1)
[[2 5 6]
 [4 7 0]
 [1 8 3]]

---
Step 6: Move the tile from (2, 2) to (1, 2)
[[2 5 6]
 [4 7 3]
 [1 8 0]]

---
Step 7: Move the tile from (2, 1) to (2, 2)
[[2 5 6]
 [4 7 3]
 [1 0 8]]

---
Step 8: Move the tile from (1, 1) to (2, 1)
[[2 5 6]
 [4 0 3]
 [1 7 8]]

---
Step 9: Move the tile from (0, 1) to (1, 1)
[[2 0 6]
 [4 5 3]
 [1 7 8]]

---
Step 10: Move the tile from (0, 0) to (0, 1)
[[0 2 6]
 [4 5 3]
 [1 7 8]]

---
Step 11: Move the tile from (1, 0) to (0, 0)
[[4 2 6]
 [0 5 3]
 [1

### A* method

### `NPuzzleSolverA`: A Solver for the N-Puzzle Problem Using A* Search

This class implements a solver for the N-Puzzle problem using the **A* Search algorithm**, which balances path cost (`g`) and heuristic cost (`h`) to find the optimal solution.

#### **Attributes**
- `start_state`: The initial puzzle state.
- `goal_state`: The target state.
- `n`: The dimension of the puzzle (e.g., `3` for a 3x3 puzzle).

#### **Methods**
1. **`is_goal(state)`**:
   - Verifies if the given `state` matches the `goal_state`.

2. **`heuristic(state)`**:
   - Calculates the **Manhattan distance** heuristic, summing the distances of each tile from its goal position (ignoring the empty tile).

3. **`search()`**:
   - Implements the **A* Search** algorithm:
     - Uses a priority queue to expand states in order of increasing `f = g + h` (path cost + heuristic).
     - Tracks visited states and their best `g` (path cost) to avoid redundant or suboptimal paths.
     - Returns:
       - `solution_path`: A sequence of moves leading to the solution.
       - `cost`: The total number of states expanded.

#### **Execution Flow**
- Start from `start_state`, iteratively expanding the state with the lowest `f`-score.
- If the goal is reached, print the solution path, quality, cost, and efficiency.
- If no solution is found, print a failure message.



In [12]:
from collections import namedtuple, defaultdict
from heapq import heappush, heappop


class NPuzzleSolverA:
    def __init__(self, start_state, goal_state):
        self.start_state = start_state
        self.goal_state = goal_state
        self.n = len(start_state)
    
    def is_goal(self, state):
        return np.array_equal(state, self.goal_state)
    
    def heuristic(self, state):
        """Calculate Manhattan distance heuristic."""
        distance = 0
        for i in range(self.n):
            for j in range(self.n):
                value = state[i, j]
                if value != 0:  # Skip the empty tile
                    goal_x, goal_y = divmod(value - 1, self.n)
                    distance += abs(i - goal_x) + abs(j - goal_y)
        return distance
    
    def search(self):
        """A* Search implementation."""
        # State wrapper for comparison in priority queue
        StateNode = namedtuple('StateNode', ['f', 'g', 'h', 'state', 'moves'])
        
        def make_state_node(state, g, moves):
            h = self.heuristic(state)
            f = g + h
            return StateNode(f=f, g=g, h=h, state=state, moves=moves)
        
        # Initialize data structures
        start_node = make_state_node(self.start_state, 0, [])
        open_set = []  # Priority queue
        heappush(open_set, (start_node.f, id(start_node), start_node))
        
        # Keep track of best g score for each state
        g_scores = defaultdict(lambda: float('inf'))
        g_scores[tuple(map(tuple, self.start_state))] = 0
        
        closed_set = set()
        expanded_states = 0
        
        while open_set:
            # Get node with lowest f-score
            _, _, current_node = heappop(open_set)
            current_state = current_node.state
            current_state_tuple = tuple(map(tuple, current_state))
            
            # Skip if we've found a better path to this state
            if current_state_tuple in closed_set:
                continue
            
            expanded_states += 1
            
            # Check if we've reached the goal
            if self.is_goal(current_state):
                return current_node.moves, expanded_states
            
            # Add to closed set
            closed_set.add(current_state_tuple)
            
            # Expand neighbors
            for move in available_actions(current_state):
                neighbor = do_action(current_state, move)
                neighbor_tuple = tuple(map(tuple, neighbor))
                
                # Calculate tentative g score
                tentative_g = current_node.g + 1
                
                # Skip if we've found a better path to this neighbor
                if tentative_g >= g_scores[neighbor_tuple]:
                    continue
                
                # This path is better than any previous one
                g_scores[neighbor_tuple] = tentative_g
                neighbor_node = make_state_node(
                    neighbor,
                    tentative_g,
                    current_node.moves + [move]
                )
                heappush(open_set, (neighbor_node.f, id(neighbor_node), neighbor_node))
        
        print("No solution found.")
        return None, expanded_states

# Example usage:
start_time = time.time()
solver = NPuzzleSolverA(state, goal)

print("Initial State:")
print(state)

solution_path, cost = solver.search()
end_time = time.time()
print(f"Execution time: {end_time - start_time:.4f} seconds")

if solution_path:
    print("Steps to solve the puzzle:")
    print_solution_steps(state, solution_path)

    # Quality and Cost
    quality = len(solution_path)
    print(f"\nQuality (number of actions in the solution): {quality}")
    print(f"Cost (total number of actions evaluated): {cost}")
    print(f"Efficiency (Quality vs Cost): {quality / cost:.4f}")

else:
    print("No solution found.")

Initial State:
[[2 5 6]
 [8 0 7]
 [4 1 3]]
Execution time: 0.0732 seconds
Steps to solve the puzzle:
Initial state:
[[2 5 6]
 [8 0 7]
 [4 1 3]]

---
Step 1: Move the tile from (1, 0) to (1, 1)
[[2 5 6]
 [0 8 7]
 [4 1 3]]

---
Step 2: Move the tile from (2, 0) to (1, 0)
[[2 5 6]
 [4 8 7]
 [0 1 3]]

---
Step 3: Move the tile from (2, 1) to (2, 0)
[[2 5 6]
 [4 8 7]
 [1 0 3]]

---
Step 4: Move the tile from (1, 1) to (2, 1)
[[2 5 6]
 [4 0 7]
 [1 8 3]]

---
Step 5: Move the tile from (1, 2) to (1, 1)
[[2 5 6]
 [4 7 0]
 [1 8 3]]

---
Step 6: Move the tile from (2, 2) to (1, 2)
[[2 5 6]
 [4 7 3]
 [1 8 0]]

---
Step 7: Move the tile from (2, 1) to (2, 2)
[[2 5 6]
 [4 7 3]
 [1 0 8]]

---
Step 8: Move the tile from (1, 1) to (2, 1)
[[2 5 6]
 [4 0 3]
 [1 7 8]]

---
Step 9: Move the tile from (1, 2) to (1, 1)
[[2 5 6]
 [4 3 0]
 [1 7 8]]

---
Step 10: Move the tile from (0, 2) to (1, 2)
[[2 5 0]
 [4 3 6]
 [1 7 8]]

---
Step 11: Move the tile from (0, 1) to (0, 2)
[[2 0 5]
 [4 3 6]
 [1 7 8]]

---
St